# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = pd.read_csv('kc_house_data.csv')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms'] * sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [7]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [8]:
from sklearn.linear_model import Lasso

In [9]:
model_all = Lasso(alpha=1e10)
model_all = model_all.fit(sales[all_features], sales['price'])

In [11]:
print model_all.coef_, model_all.intercept_

 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] 540088.141767


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
(training_and_validation, testing) = train_test_split(sales, train_size=.9, random_state=1) # initial train/test split
(training, validation) = train_test_split(training_and_validation, train_size=.5, random_state=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

In [15]:
import numpy as np
lowest_rss = float('inf')
l1_lowest_rss = 0
for l1_penalty in np.logspace(1,7,num=13):
    model = Lasso(alpha=l1_penalty)
    model = model.fit(training[all_features], training['price'])
    residual = model.predict(validation[all_features]) - validation['price']
    rss = sum(residual * residual)
    print 'rss: ' + str(rss)
    if rss < lowest_rss:
        lowest_rss = rss
        l1_lowest_rss = l1_penalty

rss: 4.16631132904e+14
rss: 4.16683655393e+14
rss: 4.1687180435e+14
rss: 4.17687366227e+14
rss: 4.20811391961e+14
rss: 4.36633951781e+14
rss: 4.47968780082e+14
rss: 4.84769265111e+14
rss: 6.00215411156e+14
rss: 6.06025586962e+14
rss: 6.27160334454e+14
rss: 6.48591957221e+14
rss: 6.55479583628e+14


In [16]:
print 'rss lowest: ' + str(lowest_rss)
print 'best l1_penalty: ' + str(l1_lowest_rss)

rss lowest: 4.16631132904e+14
best l1_penalty: 10.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [19]:
model_10 = Lasso(alpha=10.0)
model_10 = model_10.fit(testing[all_features], testing['price'])
residual_10 = testing['price'] - model_10.predict(testing[all_features])
rss = sum(residual_10 * residual_10)
print str(rss)

1.05618050065e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [20]:
model_best = Lasso(alpha=10.0)
model_best = model_best.fit(training[all_features], training['price'])
print model_best.coef_, model_best.intercept_

[ -1.53172468e+04   5.21519307e+02   5.60830419e+04   2.89790484e+02
  -4.69011296e+04   6.40966751e-01  -6.55752298e+02  -9.88875585e+04
   3.49261145e+04   4.55537885e+05   3.69020245e+04   2.85242927e+04
   1.29078141e+05   3.44275684e+02   3.46088948e+02  -3.21187479e+03
   1.57649945e+01] 6583595.36258


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [21]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [49]:
l1_penalty_values = np.logspace(7, 9, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [54]:
penalty_list = []
noweight_list = []
for l1_penalty in np.logspace(4,9,num=20):
    model = Lasso(alpha=l1_penalty)
    model = model.fit(training[all_features], training['price'])
    penalty_list.append(l1_penalty)
    noweight_list.append(np.sum(model.coef_ == 0.0))
print noweight_list
print penalty_list

[3, 5, 6, 7, 10, 11, 11, 11, 12, 13, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16]
[10000.0, 18329.807108324374, 33598.182862837813, 61584.821106602671, 112883.78916846884, 206913.80811147901, 379269.01907322538, 695192.79617756058, 1274274.9857031321, 2335721.4690901213, 4281332.3987193955, 7847599.7035146067, 14384498.882876629, 26366508.987303555, 48329302.385717519, 88586679.041008323, 162377673.91887176, 297635144.1631313, 545559478.11685145, 1000000000.0]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [55]:
l1_penalty_min = 0
l1_penalty_max = 0
for i in xrange(0, 20):
    if noweight_list[i] >= max_nonzeros:
        l1_penalty_min = penalty_list[i]
    if noweight_list[i] <= max_nonzeros:
        l1_penalty_max = penalty_list[i]
        break
print 'l1_penalty_min: ' + str(l1_penalty_min)
print 'l1_penalty_max: ' + str(l1_penalty_max)

l1_penalty_min: 0
l1_penalty_max: 10000.0


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [67]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [71]:
lowest_rss = float('inf')
bestmodel = 0
best_l1 = 0
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    residual = model.predict(validation) - validation['price']
    rss = sum(residual * residual)
    if (rss <= lowest_rss) & (model['coefficients']['value'].nnz() == max_nonzeros):
        bestmodel = model
        best_l1 = l1_penalty
        lowest_rss = rss

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [72]:
print "best_l1: " + str(best_l1)
print "lowest_rss: " + str(lowest_rss)
bestmodel.get('coefficients').print_rows(18,)

best_l1: 3448968612.16
lowest_rss: 1.04693748875e+15
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 